In [25]:
import tweepy
import json
import pandas as pd

pd.set_option('display.max_colwidth', -1)
# == OAuth Authentication ==
#
# This mode of authentication is the new preferred way
# of authenticating with Twitter.

secret = json.load(open('secret.json'))

# The consumer keys can be found on your application's Details
# page located at https://dev.twitter.com/apps (under "OAuth settings")
consumer_key=secret["consumer_key"]
consumer_secret=secret["consumer_secret"]

# The access tokens can be found on your applications's Details
# page located at https://dev.twitter.com/apps (located
# under "Your access token")
access_token=secret["access_token"]
access_token_secret=secret["access_token_secret"]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# If the authentication was successful, you should
# see the name of the account print out
print(api.me())


# status = api.get_status(id, tweet_mode="extended")
# status

# a=api.search(q="bolivia",count=100,tweet_mode="extended",result_type="recent")


# b=api.

# If the application settings are set for "Read and Write" then
# this line should tweet out the message to your account's
# timeline. The "Read and Write" setting is on https://dev.twitter.com/apps
#api.update_status(status='Updating using OAuth authentication via Tweepy!')

{'id': 753865475533275137, 'id_str': '753865475533275137', 'name': 'Jorge Antequera', 'screen_name': 'JorgeAntequera_', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 14, 'friends_count': 178, 'listed_count': 0, 'created_at': 'Fri Jul 15 08:15:10 +0000 2016', 'favourites_count': 8, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 4, 'lang': None, 'status': {'created_at': 'Tue Nov 12 23:21:48 +0000 2019', 'id': 1194394895055773696, 'id_str': '1194394895055773696', 'text': 'RT @Almagro_OEA2015: ¿Golpe de Estado en #Bolivia?\n\nSí. En el momento que pretendieron robarse las elecciones del 20 de octubre y perpetuar…', 'truncated': False, 'entities': {'hashtags': [{'text': 'Bolivia', 'indices': [41, 49]}], 'symbols': [], 'user_mentions': [{'screen_name': 'Almagro_OEA2015', 'name': 'Luis Almagro', 'id': 2802545808, 'id_str': '2802545808

In [26]:
def transform(tweet_query_json):
    l=[]
    for tweet in tweet_query_json["statuses"]:
        try:
            l.append((tweet["retweeted_status"]["created_at"],\
                      tweet["retweeted_status"]["full_text"],tweet["retweeted_status"]["id"], \
                      tweet["id"],
                      tweet["retweeted_status"]["user"]["name"],tweet["retweeted_status"]["retweet_count"],\
                      tweet["retweeted_status"]["favorite_count"],tweet["retweeted_status"]["user"]["followers_count"],\
                      tweet["retweeted_status"]["user"]["friends_count"]))   
        except:
            l.append((tweet["created_at"],tweet["full_text"],tweet["id"],tweet["id"],tweet["user"]["name"],tweet["retweet_count"],\
                      tweet["favorite_count"],tweet["user"]["followers_count"],tweet["user"]["friends_count"]))


    df = pd.DataFrame(l,columns=["created_at","text","tweet_id","tweet_original_id","original_user","retweets","favs","followers","friends"])

    df = df.drop_duplicates(subset="tweet_id").reset_index(drop=True)
    
    min_id = df["tweet_original_id"].min()
    
    return [min_id,df]

def search_tweet(topic,max_id):
    json_tweet = api.search(q=topic,count=1000,tweet_mode="extended",result_type="recent",max_id=max_id,wait_on_rate_limit=True)
    return json_tweet



tup = transform(search_tweet("bolivia",'5'))


In [ ]:
# max_id=1191108460311728130
# api.search(q="bolivia",count=1000,tweet_mode="extended",result_type="recent",max_id=max_id,wait_on_rate_limit=True)

In [27]:
import time

while (tup[1].shape[0]<50000):
#     print(tup[0])
    df_temp = transform(search_tweet("bolivie",max_id=tup[0]))[1] 
    tup[0] = df_temp["tweet_original_id"].min()
    tup[1] = tup[1].append(df_temp)\
    .drop_duplicates(subset="tweet_id")\
    .reset_index(drop=True)
    time.sleep(0.1)
    print(tup[0])

1196476730376171525
1196470465608454145
1196466033634299904
1196458180739502082
1196446895499071490
1196434869859110912


KeyboardInterrupt: 

In [28]:
# search_tweet("bolivia",max_id=1190432381230047237)
# transform(search_tweet("bolivia",max_id=tup[0]))[1]
# transform(search_tweet("bolivia",max_id=tup[0]))[1]["tweet_original_id"].min()
# df_temp = transform(search_tweet("bolivia",max_id=1191108460311728130))[1] 
# df_temp["tweet_original_id"].sort_values()
# df_temp2[["tweet_id","tweet_original_id"]].sort_values(by="tweet_id")
df = tup[1]
df

created_at  \
0    Mon Nov 18 16:36:00 +0000 2019   
1    Sun Nov 17 22:55:43 +0000 2019   
2    Mon Nov 18 17:32:24 +0000 2019   
3    Mon Nov 18 17:29:52 +0000 2019   
4    Mon Nov 18 14:25:41 +0000 2019   
5    Mon Nov 18 17:16:53 +0000 2019   
6    Mon Nov 18 17:27:14 +0000 2019   
7    Mon Nov 18 17:31:27 +0000 2019   
8    Mon Nov 18 10:01:06 +0000 2019   
9    Mon Nov 18 17:31:11 +0000 2019   
10   Mon Nov 18 17:30:50 +0000 2019   
11   Mon Nov 18 17:26:00 +0000 2019   
12   Mon Nov 18 13:20:52 +0000 2019   
13   Mon Nov 18 08:18:15 +0000 2019   
14   Mon Nov 18 16:34:22 +0000 2019   
15   Mon Nov 18 15:47:40 +0000 2019   
16   Mon Nov 18 17:29:54 +0000 2019   
17   Mon Nov 18 11:35:40 +0000 2019   
18   Mon Nov 18 09:35:05 +0000 2019   
19   Mon Nov 18 17:28:41 +0000 2019   
20   Mon Nov 18 17:28:11 +0000 2019   
21   Sun Nov 17 20:28:35 +0000 2019   
22   Mon Nov 18 17:27:42 +0000 2019   
23   Mon Nov 18 17:26:49 +0000 2019   
24   Mon Nov 18 17:26:35 +0000 2019   
25   Sun Nov 17 20:40:36 +0000 2019   
26   Mon Nov 18 09:42:00 +0000 2019   
27   Sun Nov 17 14:25:21 +0000 2019   
28   Mon Nov 18 16:00:47 +0000 2019   
29   Mon Nov 18 17:23:02 +0000 2019   
..                              ...   
198  Mon Nov 18 14:27:56 +0000 2019   
199  Mon Nov 18 14:26:37 +0000 2019   
200  Mon Nov 18 14:23:07 +0000 2019   
201  Mon Nov 18 13:41:50 +0000 2019   
202  Mon Nov 18 14:24:16 +0000 2019   
203  Mon Nov 18 14:24:15 +0000 2019   
204  Mon Nov 18 14:21:45 +0000 2019   
205  Mon Nov 18 14:16:33 +0000 2019   
206  Mon Nov 18 14:14:39 +0000 2019   
207  Mon Nov 11 13:40:52 +0000 2019   
208  Mon Nov 18 14:13:01 +0000 2019   
209  Mon Nov 18 14:12:52 +0000 2019   
210  Mon Nov 18 14:12:46 +0000 2019   
211  Mon Nov 18 03:05:48 +0000 2019   
212  Mon Nov 18 14:10:25 +0000 2019   
213  Mon Nov 18 14:09:52 +0000 2019   
214  Mon Nov 18 14:07:35 +0000 2019   
215  Mon Nov 18 14:07:23 +0000 2019   
216  Mon Nov 18 14:07:21 +0000 2019   
217  Mon Nov 18 14:07:13 +0000 2019   
218  Mon Nov 18 13:40:47 +0000 2019   
219  Sun Nov 17 15:30:20 +0000 2019   
220  Mon Nov 18 14:03:27 +0000 2019   
221  Mon Nov 18 14:02:32 +0000 2019   
222  Mon Nov 18 13:58:19 +0000 2019   
223  Sat Nov 16 09:38:49 +0000 2019   
224  Mon Nov 18 13:55:20 +0000 2019   
225  Mon Nov 18 13:55:16 +0000 2019   
226  Mon Nov 18 13:53:12 +0000 2019   
227  Mon Nov 18 13:49:45 +0000 2019   

                                                                                                                                                                                                                                                                                                                       text  \
0    Les coups d’État sont l'aboutissement d'une longue lutte entre les forces de l'impérialisme et de la décolonisation. Mieux vaut laisser les généraux faire le sale boulot, tandis que l'ambassade des États-Unis reste sans tache. https://t.co/MBW08lztPf                                                               
1    Bolivie : le "blocage" des pro-Morales provoque des pénuries à La Paz https://t.co/R8vqnGdd9r https://t.co/0EjxhhZwxl                                                                                                                                                                                                    
2    Bolivie : Coup d’état raciste contre ses peuples indigènes ! Par Yorgos Mitralias - Le blog de algerie-infos https://t.co/mdcUmkVeGF                                                                                                                                                                                     
3    Alla fine, dopo attenti studi, a Le Monde sorge un terribile dubbio: vuoi vedere che era un golpe? \n\nMedia borghesi semplicemente allucinanti e pozzi senza fondo di malafede e ipocrisia https://t.co/Z1vIEY0wNn                                                                                                      
4    Les fe

In [29]:
# tup[1].sort_values(by="followers",ascending=False
# df_temp2

df["split"]=(df["text"].str.lower()
             .str.replace("\n"," ")
             .str.replace("[\.\,\?\!\:\"\”\“\¿\-\🇧🇴\|]","")
             .str.replace("\s{2,}"," ")
             .str.strip()
             .str.split(" "))

df.sort_values("favs",ascending=False)

created_at  \
21   Sun Nov 17 20:28:35 +0000 2019   
223  Sat Nov 16 09:38:49 +0000 2019   
27   Sun Nov 17 14:25:21 +0000 2019   
46   Sun Nov 17 18:00:36 +0000 2019   
191  Mon Nov 11 09:56:25 +0000 2019   
132  Mon Nov 11 01:12:36 +0000 2019   
155  Sun Nov 17 11:47:19 +0000 2019   
207  Mon Nov 11 13:40:52 +0000 2019   
18   Mon Nov 18 09:35:05 +0000 2019   
129  Sun Nov 17 11:18:47 +0000 2019   
26   Mon Nov 18 09:42:00 +0000 2019   
127  Sun Nov 17 12:32:54 +0000 2019   
25   Sun Nov 17 20:40:36 +0000 2019   
30   Mon Nov 18 07:48:01 +0000 2019   
120  Sun Nov 17 19:29:56 +0000 2019   
109  Sun Nov 17 14:20:37 +0000 2019   
13   Mon Nov 18 08:18:15 +0000 2019   
14   Mon Nov 18 16:34:22 +0000 2019   
91   Thu Nov 14 14:37:24 +0000 2019   
161  Sun Nov 17 13:59:40 +0000 2019   
133  Wed Nov 13 11:38:43 +0000 2019   
110  Sun Nov 17 19:39:18 +0000 2019   
83   Mon Nov 18 06:34:46 +0000 2019   
72   Mon Nov 18 12:25:40 +0000 2019   
168  Sun Nov 17 17:29:21 +0000 2019   
39   Sun Nov 17 17:46:26 +0000 2019   
67   Mon Nov 18 09:34:43 +0000 2019   
219  Sun Nov 17 15:30:20 +0000 2019   
70   Mon Nov 18 12:31:55 +0000 2019   
176  Mon Nov 18 08:08:58 +0000 2019   
..                              ...   
134  Mon Nov 18 11:56:50 +0000 2019   
135  Mon Nov 18 15:47:11 +0000 2019   
137  Mon Nov 18 15:43:48 +0000 2019   
139  Mon Nov 18 15:42:45 +0000 2019   
141  Mon Nov 18 15:35:58 +0000 2019   
88   Mon Nov 18 16:39:10 +0000 2019   
143  Mon Nov 18 15:37:11 +0000 2019   
144  Mon Nov 18 15:34:38 +0000 2019   
145  Mon Nov 18 15:32:28 +0000 2019   
146  Mon Nov 18 15:31:35 +0000 2019   
151  Mon Nov 18 15:25:52 +0000 2019   
152  Mon Nov 18 15:16:08 +0000 2019   
123  Mon Nov 18 16:00:11 +0000 2019   
119  Mon Nov 18 16:01:37 +0000 2019   
116  Mon Nov 18 16:04:35 +0000 2019   
115  Mon Nov 18 16:05:52 +0000 2019   
111  Mon Nov 18 16:13:24 +0000 2019   
108  Mon Nov 18 16:17:30 +0000 2019   
107  Mon Nov 18 16:19:05 +0000 2019   
106  Mon Nov 18 16:19:37 +0000 2019   
104  Mon Nov 18 16:23:10 +0000 2019   
103  Mon Nov 18 16:23:40 +0000 2019   
101  Mon Nov 18 16:24:03 +0000 2019   
100  Mon Nov 18 16:24:27 +0000 2019   
99   Mon Nov 18 16:25:13 +0000 2019   
94   Mon Nov 18 15:30:00 +0000 2019   
93   Mon Nov 18 16:33:06 +0000 2019   
92   Mon Nov 18 16:36:22 +0000 2019   
89   Mon Nov 18 16:38:25 +0000 2019   
227  Mon Nov 18 13:49:45 +0000 2019   

                                                                                                                                                                                                                                                                                                                                 text  \
21   Antoine Griezmann n'avait plus marqué avec les Bleus depuis le 2 juin dernier (contre la Bolivie), soit depuis 708 minutes de jeu. 🇫🇷👊 https://t.co/peBzpGEtPi                                                                                                                                                                     
223  Ce matin sur France inter, on entend parler de Jeanine Áñez comme «présidente de Bolivie», sans nuance sur le fait qu'elle est autoproclamée, sans mention du coup d'État. \n\nFranchement ! Un peu de sérieux ! \n\nOn paye pour un service public de l'info, pas pour Radio CIA !                                                
27   En #Bolivie, la Présidente auto-proclamée fait tirer sur les manifestants pro-Morales et autorise l'armée à participer au maintien de l'ordre en l'exonérant de toutes responsabilités pénales. Qu'attendent la France et l'Union européenne pour enfin réagir ? https://t.co/8aVK2zxIpl                                           
46   Mobilisation à #Paris pour protester contre le coup d'État en #Bolivie et réaffirmer notre solidarité avec #EvoMorales .Le peuple bolivien est aujourd'hui en danger face aux racistes et fondamentalistes qui prétendent en finir avec les amérindiens ! #Solidarité #EvoNoEstasSolo https:/